In [1]:
'''
Check improved topic representation for selected model
'''

'\nCheck improved topic representation for selected model\n'

In [2]:
!pip install stopwordsiso

In [3]:
from stopwordsiso import stopwords
stopwords_nl = list(stopwords('nl'))

In [ ]:
#import libraries

import numpy as np
import os
from bertopic import BERTopic
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#specify path to data files
csv_sample_file = "/home/tom/Documents/code/GitHub/dutch-elections/DATA/Parties_and_LeadersV2.csv"

#load data
print('load data')
telegram_df = pd.read_csv(csv_sample_file)

docs = list(telegram_df['text']) #get list of messages
sample_size = print('number of docs', len(docs))

#load embeddings
embedding = np.load("/home/tom/Documents/code/GitHub/dutch-elections/embeddings/paraphrase-multilingual-mpnet-base-v2.npy")

/home/tom/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


load data
number of docs 22499


In [ ]:
model_folder = "BERTopic_models/elections"

for model_file in tqdm(os.listdir(model_folder)):

    if not model_file.endswith('reduced_outliers'):
        continue

    #load model
    print('load topic model')
    topic_model = BERTopic.load(os.path.join(model_folder, model_file), embedding_model = embedding)


    #set parameters for topic labels
    vectorizer_model = CountVectorizer(ngram_range=(1, 2), max_df = 0.7, stop_words= stopwords_nl)
    topic_model.update_topics(docs, vectorizer_model=vectorizer_model)

    topic_labels = topic_model.generate_topic_labels(nr_words=50,
                                                    topic_prefix=True,
                                                    word_length=50,
                                                    separator=", ")

    print('update topic model labels')
    topic_model.set_topic_labels(topic_labels)

    #save the updated topic model and topic model information
    print('save updated topic model and topic model information')
    updated_topic_model_directory = os.path.join(model_folder, model_file + '_updated_keywords')
    if not os.path.isdir(updated_topic_model_directory):
        print('create topic model folder:', updated_topic_model_directory)
        os.mkdir(updated_topic_model_directory)

    print('save updated topic model')
    embedding_model = SentenceTransformer("")
    topic_model.save(updated_topic_model_directory, serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

    #check new topics
    topic_model_info = topic_model.get_topic_info()

    #store topic model info for inspection
    print('store info')
    topic_model_info.to_csv(os.path.join(model_folder, model_file + '__updated_keywords.csv'))


  0%|          | 0/7 [00:00<?, ?it/s]

load topic model
update topic model labels
save updated topic model and topic model information
create topic model folder: BERTopic_models/paraphrase-multilingual-mpnet-base-v2_topic_model_50_reduced_outliers_updated_keywords
save updated topic model


 29%|██▊       | 2/7 [00:06<00:16,  3.37s/it]

store info
load topic model
update topic model labels
save updated topic model and topic model information
create topic model folder: BERTopic_models/paraphrase-multilingual-mpnet-base-v2_topic_model_60_reduced_outliers_updated_keywords
save updated topic model


100%|██████████| 7/7 [00:13<00:00,  1.94s/it]

store info


In [ ]:
#inspect the topic model
import numpy as np
from bertopic import BERTopic

print('load topic model')
embedding = np.load("/home/tom/Documents/code/GitHub/dutch-elections/embeddings/paraphrase-multilingual-mpnet-base-v2.npy")
topic_model = BERTopic.load("home/tom/Documents/code/GitHub/dutch-elections/BERTopic_models/paraphrase-multilingual-mpnet-base-v2_topic_model_70_reduced_outliers_updated_keywords", embedding_model = embedding)
topic_info = topic_model.get_topic_info()

In [ ]:
#fit topics to documents
print('fit docs to topics')
topics = topic_model.topics_  
doc_to_topic_df = pd.DataFrame({"document": docs, "topic": topics})
doc_to_topic_df.to_csv('doc_to_topic.csv')

In [ ]:
doc_to_topic_df